<a href="https://colab.research.google.com/github/elisasmenendez/base-caatinga/blob/main/Caatinga_Species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

As Redes Neurais Convolucionais são mais adequadas para classificação de imagens. Nesta estratégia não é necessário utilizar todos os pixels da imagem pois são utilizados filtros para reduzir a dimensionalidade das imagens.

In [ ]:
!pip install split-folders
#!pip install tensorflow==2.6.0

In [ ]:
# Baixando o arquivo diretamente do repositório do Git
!wget https://github.com/elisasmenendez/base-caatinga/raw/refs/heads/main/Dataset.zip

# Extrair o Dataset.zip com suporte a acentos
!unzip -O ISO-8859-1 Dataset.zip

In [ ]:
import os
import splitfolders
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np

# Treinamento e Teste

In [ ]:
dataset_dir = "Dataset"

for class_name in sorted(os.listdir(dataset_dir)):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        num_images = len([
            fname for fname in os.listdir(class_path)
            if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        print(f"{class_name}: {num_images} imagens")

Facheiro: 19 imagens
Jericó: 27 imagens
Macambira-de-flecha: 50 imagens
Marmeleiro: 20 imagens
Palmatória: 50 imagens
Quipá: 50 imagens
Xique-Xique: 50 imagens


In [ ]:
splitfolders.ratio("Dataset", output="Dataset_split", seed=42, ratio=(.8, .2), move=False)

# Data augmentation

In [ ]:
import unicodedata

def normalize(text):
    return unicodedata.normalize("NFC", text)

In [ ]:
fixed_class_names = [
    "Facheiro",
    "Jericó",
    "Macambira-de-flecha",
    "Marmeleiro",
    "Palmatória",
    "Xique-Xique",
    "Quipá",
]

label_to_index = {
    normalize(name): idx for idx, name in enumerate(fixed_class_names)
}

In [ ]:
label_to_index

{'Facheiro': 0,
 'Jericó': 1,
 'Macambira-de-flecha': 2,
 'Marmeleiro': 3,
 'Palmatória': 4,
 'Xique-Xique': 5,
 'Quipá': 6}

In [ ]:
def remap_labels(images, labels):
    class_names = train_ds_raw.class_names  # ordem automática do TensorFlow
    remapped_labels = tf.convert_to_tensor([
        label_to_index[normalize(class_names[label])] for label in labels.numpy()
    ])
    return images, remapped_labels

# Usar .map() com tf.py_function porque labels.numpy() é necessário
def tf_remap(images, labels):
    remapped_images, remapped_labels = tf.py_function(
        func=remap_labels,
        inp=[images, labels],
        Tout=(tf.float32, tf.int32)
    )
    remapped_images.set_shape((None, 224, 224, 3))
    remapped_labels.set_shape((None,))
    return remapped_images, remapped_labels

In [ ]:
train_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    "Dataset_split/train",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int"
)

val_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    "Dataset_split/val",
    image_size=(224, 224),
    batch_size=32,
    label_mode="int"
)

# Salva a ordem original das classes detectadas automaticamente
original_class_names = train_ds_raw.class_names

# Deixar com a ordem certa
train_ds = train_ds_raw.map(tf_remap)
val_ds = val_ds_raw.map(tf_remap)

# Data augmentation (apenas treino)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)
])

AUTOTUNE = tf.data.AUTOTUNE

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=AUTOTUNE
).prefetch(AUTOTUNE)

val_ds = val_ds.prefetch(AUTOTUNE)

# Calcular pesos das classes
all_labels = []
for _, labels in train_ds:
    all_labels.extend(labels.numpy())

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(all_labels),
    y=all_labels
)

class_weights_dict = dict(enumerate(class_weights))

# Mostrar pesos das classes
df = pd.DataFrame({
    "Classe": fixed_class_names,
    "Peso": [class_weights_dict[i] for i in range(len(fixed_class_names))]
})
print(df)

Found 212 files belonging to 7 classes.
Found 54 files belonging to 7 classes.
                Classe      Peso
0             Facheiro  2.019048
1               Jericó  1.442177
2  Macambira-de-flecha  0.757143
3           Marmeleiro  1.892857
4           Palmatória  0.757143
5          Xique-Xique  0.757143
6                Quipá  0.757143


# Construção e treinamento da rede neural

In [ ]:
# Número de classes do dataset
num_classes = len(fixed_class_names)

# Modelo baseado na MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Congela a base pré-treinada

# Construção do modelo final
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Normalização
    data_augmentation,                 # Aumento de dados (apenas treino)
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compilar o modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Exibir o resumo do modelo
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_9 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
model.fit(
    augmented_train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights_dict
)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.2071 - loss: 2.2018 - val_accuracy: 0.7407 - val_loss: 1.0006
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.7211 - loss: 0.8675 - val_accuracy: 0.8704 - val_loss: 0.6209
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.7955 - loss: 0.5148 - val_accuracy: 0.8519 - val_loss: 0.4236
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.8755 - loss: 0.3845 - val_accuracy: 0.8519 - val_loss: 0.3596
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.9250 - loss: 0.2590 - val_accuracy: 0.8704 - val_loss: 0.3167
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9258 - loss: 0.2211 - val_accuracy: 0.8519 - val_loss: 0.3467
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.8988 - loss: 0.2001 - val_accuracy: 0.8889 - val_loss: 0.3105
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 24s 3s/step - accuracy: 0.9306 - loss: 0.1731 - val_accuracy: 0.8889 - val_loss: 0.2876
Epoch 9/

# Avaliação da rede neural

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# 1. Obter previsões do modelo
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    predicted_classes = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(predicted_classes)

# 2. Calcular accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.9259


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=fixed_class_names))

[[ 3  0  0  0  0  1  0]
 [ 0  6  0  0  0  0  0]
 [ 0  0 10  0  0  0  0]
 [ 0  0  0  4  0  0  0]
 [ 0  0  0  0 10  0  0]
 [ 0  0  0  0  0  9  1]
 [ 0  0  0  0  2  0  8]]
                     precision    recall  f1-score   support

           Facheiro       1.00      0.75      0.86         4
             Jericó       1.00      1.00      1.00         6
Macambira-de-flecha       1.00      1.00      1.00        10
         Marmeleiro       1.00      1.00      1.00         4
         Palmatória       0.83      1.00      0.91        10
        Xique-Xique       0.90      0.90      0.90        10
              Quipá       0.89      0.80      0.84        10

           accuracy                           0.93        54
          macro avg       0.95      0.92      0.93        54
       weighted avg       0.93      0.93      0.92        54



In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# 1. Carrega a imagem externa
img_path = "/content/Xique-Xique.png"  # ajuste o caminho
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# 2. (Opcional) Normalização — SOMENTE se o modelo não tiver Rescaling
# img_array = img_array / 255.0

# 3. Faz a predição
prediction = model.predict(img_array)  # shape (1, num_classes)
predicted_index = np.argmax(prediction)
confidence = prediction[0][predicted_index]

# 4. Pega o nome da classe
predicted_class = fixed_class_names[predicted_index]

# 5. Mostra o resultado
print(f"Classe prevista: {predicted_class}")
print(f"Confiança: {confidence * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Classe prevista: Xique-Xique
Confiança: 99.83%


# Salvando o modelo TFLITE

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Conversão com quantização para uint8 (compatível com TFLite do Teachable Machine)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Precisa de um conjunto de calibração para inferir escala/zero-point
def representative_data_gen():
    for images, _ in train_ds.take(100):  # pode usar uma amostra
        yield [tf.cast(images, tf.float32)]

converter.representative_dataset = representative_data_gen

tflite_model = converter.convert()

Saved artifact at '/tmp/tmpies05r3b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_352')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  132580323708944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323706256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323711824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323709904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323719696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323710288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323706832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323710864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323710096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132580323720080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1325803237

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
# Salva no disco
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
with open("labels.txt", "w", encoding="utf-8") as f:
    for idx, label in enumerate(fixed_class_names):
        f.write(f"{idx} {label}\n")

# Testando o TFLite

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image

# === 1. Carregar o modelo .tflite ===
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# === 2. Carregar os rótulos (labels.txt com índices fixos) ===
labels = []
with open("labels.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split(" ", 1)
        labels.append(parts[1] if len(parts) > 1 else parts[0])

# === 3. Detalhes do modelo ===
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']      # Ex: [1, 224, 224, 3]
input_dtype = input_details[0]['dtype']      # Geralmente uint8 ou float32
output_scale, output_zero_point = output_details[0]['quantization']

# === 4. Pré-processamento da imagem ===
def preprocess_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((224, 224))
    img_array = np.array(img)

    if input_dtype == np.uint8:
        return np.expand_dims(img_array.astype(np.uint8), axis=0)
    else:
        img_array = img_array.astype(np.float32) / 255.0
        return np.expand_dims(img_array, axis=0)

# === 5. Classificação ===
def classify_image(img_path):
    input_data = preprocess_image(img_path)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Obter saída bruta e desscalar
    raw_output = interpreter.get_tensor(output_details[0]['index'])[0]
    output_data = output_scale * (raw_output.astype(np.float32) - output_zero_point)

    predicted_index = np.argmax(output_data)
    confidence = output_data[predicted_index]

    print(f"Classe prevista: {labels[predicted_index]}")
    print(f"Confiança: {confidence * 100:.2f}%")

In [ ]:
# === 6. Testar ===
classify_image("/content/Teste1.jpg")  # troque pelo nome da sua imagem

Classe prevista: Xique-Xique
Confiança: 82.03%
